In [1]:
from pyspark.sql import SparkSession
from pyspark.sql.functions import *

In [2]:
spark = SparkSession.builder.appName("films").getOrCreate()

In [6]:
movies_df = spark.read.csv("data/films_dir/movies.csv", header=True, inferSchema=True)
actors_df = spark.read.csv("data/films_dir/actors.csv", header=True, inferSchema=True)
movie_actors_df = spark.read.csv("data/films_dir/movie_actors.csv", header=True, inferSchema=True)

Создание временных таблиц

In [9]:
movies_df.createOrReplaceTempView("movies")
actors_df.createOrReplaceTempView("actors")
movie_actors_df.createOrReplaceTempView("movie_actors")

Топ-5 жанров по количеству фильмов

In [15]:
top_5_genres = spark.sql("""
SELECT genre, count(*) as num_movies FROM movies
    GROUP BY genre
    ORDER BY num_movies DESC
    LIMIT 5
""")
top_5_genres.show()

+------+----------+
| genre|num_movies|
+------+----------+
| Drama|         6|
|Action|         6|
|Comedy|         4|
|Horror|         2|
|Sci-Fi|         2|
+------+----------+



Актер с наибольшим количеством фильмов

In [19]:
top_actor = spark.sql("""
SELECT a.name, COUNT(*) as num_movies
    FROM actors a
        JOIN movie_actors ma 
        ON a.actor_id = ma.actor_id
    GROUP BY a.name
    ORDER BY num_movies DESC
    LIMIT 1
""")

top_actor.show()

+--------+----------+
|    name|num_movies|
+--------+----------+
|Actor_17|         5|
+--------+----------+



Средний бюджет фильмов по жанрам

In [20]:
avg_budget_by_genre = spark.sql("""
SELECT genre, AVG(budget) as avg_budget
FROM movies
GROUP BY genre
""")

avg_budget_by_genre.show()

+------+--------------------+
| genre|          avg_budget|
+------+--------------------+
| Drama| 6.076021856166667E7|
|Horror|      8.7281876775E7|
|Comedy|     5.20709662225E7|
|Action|2.7492742561666667E7|
|Sci-Fi|       7.809715175E7|
+------+--------------------+



Фильмы с более чем одним актером из одной страны

In [22]:
movies_with_multiple_actors_same_country = spark.sql("""
SELECT m.title, a.country, COUNT(*) as num_actors
FROM movies m
JOIN movie_actors ma ON m.movie_id = ma.movie_id
JOIN actors a ON ma.actor_id = a.actor_id
GROUP BY m.title, a.country
HAVING num_actors > 1
""")

movies_with_multiple_actors_same_country.show()

+--------+---------+----------+
|   title|  country|num_actors|
+--------+---------+----------+
| Movie_7|    India|         2|
| Movie_3|      USA|         2|
|Movie_10|       UK|         2|
|Movie_15|    India|         2|
|Movie_18|Australia|         2|
| Movie_1|    India|         3|
| Movie_2|      USA|         2|
| Movie_7|      USA|         2|
|Movie_10|      USA|         2|
+--------+---------+----------+



In [23]:
spark.stop()